<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/05_%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%82%92%E5%88%A9%E7%94%A8%E3%81%97%E3%81%9F%E7%89%A9%E4%BD%93%E6%A4%9C%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出

点群の物体検出は、点群中に存在する物体の位置・サイズ・向きを推定するタスクです。
物体検出の使われる状況は、自動運転の歩行者検出や車の検出であり、リアルタイムな処理が必要です。


## 環境設定

In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

CUDAの設定

In [ ]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.8/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.8/lib64:{ld}"
!nvcc --version

### ライブラリのインストール

CUDAが認識されているかを確認する

In [ ]:
!nvidia-smi

In [ ]:
!pip install open3d

### Open3Dの機械学習機能をインストール

Open3Dには、機械学習、特に深層学習手法の学習済みモデルが公開されています。

In [ ]:
!git clone https://github.com/isl-org/Open3D-ML.git

Open3Dの機械学習機能は自分でビルドする必要があります。

まず、ディレクトリへ移動します。

In [ ]:
%cd Open3D-ML

必要ライブラリを確認する。

In [ ]:
!cat requirements-torch-cuda.txt

必要なライブラリをインストールする。

In [ ]:
# !pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111
!pip install -r requirements-torch-cuda.txt
!pip install -r requirements-tensorflow.txt

## 学習済みモデルによる推論

ここでは、深層学習による物体検出手法であるpointpillarsの学習済みモデルを用います。
pointpillarsは、pointnetを物体検出向けに拡張した手法です。

#### 概要
一般的に，点群のみ使用したネットワークは精度が高いです．しかし低速という問題があります．一方，点群を画像に投影するネットワークは高速だが精度が低いという欠点があった。
そこで，Point Pillarsは点群の細かい情報量を失わないように情報量をエンコードし、疑似画像に変換する。その疑似画像を2D CNNで使用するような物体検出ネットワークに入力し、物体検出を行う。 
この手法の進歩性は従来単純に点群を俯瞰画像といった疑似画像に投影し物体検出CNNに入力するだけでは点群の細かい情報量が失われてしまっていた。そこで点群を画像に投影するためのエンコードネットワークを用いることで点群情報を失わずに物体検出CNN（SSD)に入力データを与え高精度化を達成した。
具体的に点群を画像にエンコードするために、Pillar(柱）と呼ぶ点群を細かく格子状に分割しPointNetのような点群DNNを使いPillar内の特徴量を抽出。そして得た2Dの特徴量マップをSSDに与えることで物体検出を行う。 
アイデア自体は非常にシンプルながら、PointNetと物体検出CNNに結合する手法です。

#### 層設計
VoxelNetではVoxel単位でPointNetを用いて特徴量を算出してから画像に投影し物体検出を行いました．
Point Pillarsでは，以下の手順でEnd-to-Endに物体検出を行います．
1.	Pillar Feature Net: 点群をスパースなpseudo imageに変換する
2.	Backbone (2D CNN): pseudo imageをハイレベル特徴量へ変換する
3.	Detection Head(SSD): 検出と３Dボックスへの回帰を行う
 



この３つの手順の中で重要なのが「Pillar Feature Net」なので，詳細に説明します．
まず，点群をx, y平面でPillarを用いて分割します．
この際，Pillar内部に入る点に対してpillar内の点の平均値からの差とpillar中心からの距離を計算し，新たな特徴量とします．
次に，Pillar内の点群に対してPointNetを適用して特徴量を計算します．
作成した特徴量を2次元画像に投影することで，擬似画像を作成します．
この後に，通常の画像に対する物体検出手法を適用すれば，車や歩行者等の物体が検出できます．
 

#### ピラーの作成方法
入力次元では，絶対座標（x, y, z）, 反射強度rの4チャンネルの他に，ピーラー内部の相対的な座標情報（ピラー内の平均値からの差 (x_c, y_c, z_c)とピラー中心からの距離(x_p, y_p)）を作成し，9次元とします．
ピラーの集合はDxPxNの次元を持つ．
P: 点の入ったピラーの数（空っぽのピラーは無視する）
N: ピラー内の点の数
D: 入力次元であれば9，途中の層であればチャンネル数
※ピラー内部にN点以上あった場合は，ランダムにサンプリングしてN点抽出する．逆に，N点より少ない場合は，Nになるように残りをゼロ埋めする．なお，入力順番はNの階乗だけバリエーションがありますが，後段の処理で順番依存が抜けるので気にしないで大丈夫です．

これは，1x1の畳み込みを適用すると，Nの方向には独立しつつD方向に全結合のMLPと等価となり，高速な演算が可能になります．

ピラー単位の特徴抽出によってCxPxNのテンソルができたとします．なお，C
は特徴マップがCチャンネルの意味です．ここから，ピラー内の代表値を推定します．代表値にはピラー内に存在するN点に対する特徴マップのMaxPoolingによって得られた値を用います．ピラーの代表値ができため，CxPのテンソルができます．
この後は鳥瞰図の画像に投影します．この時は，ピラーのIDとxy座標の関係を保持した情報を元にグリッド画像（CxHxW）に並べなおします．HとWは画像の高さと幅です．

#### 物体に応じたネットワーク構造が必要
ピラーを作成する際に，物体の大きさによって物体検出を行う際のバックボーンネットワークのストライドのサイズを変更する必要がある．例えば，歩行者と自転車は小さなストライドで抽出できるが，車を抽出するには大きなストライドが必要です．小さなストライドでは車の特徴を作成できず，逆に大きなストライドでは歩行者の微細な特徴を消してしまう可能性があります．

#### データ拡張
１．	ランダムに配置
もともと観測された点群だけでなく，ランダムに物体を配置します．
２．	物体単位の拡張
ランダムに各物体をx,y,z方向に-9~９度回転させたり，x,y,z座標をシフトさせたりします．
３．	全体的な拡張
道路の傾きをデータ拡張で得るために，点群全体に傾き加えます．また，x,y,z座標に全体的なシフトをかけます．

#### loss
Loss設計は物体のある座標(x,y,z)，ボックスの大きさ（w, l, h），回転(θ)の値を回帰で最小化します．各 x,y,z, w,l,h, θの真値と推定結果の差は以下のように定義します．なお，サイズによるlossへの寄与度を正規化するため，サイズ由来のパラメータで割る処理を入れています．
 
 
 

なお，Smooth L1 lossは通常のL1距離よりも，真値に近い場合には誤差の値を小さくするような機能を持ちます．定義は以下のようになります．
 

この際，回転角の回帰は0 and πラジアンで計算するため，前を向いているのか後ろを向いているのかの区別つかないような現象が発生します．そこで，どの向きなのかを分類するloss(L_dir)を加えています．（著者の実装では，前か後ろの二値分類）

また，物体検出は物体のある場所だけでなく，クラスも推定する必要があるため，画像の物体検出で用いられるFocal lossを計算します．
 

最終的なlossはこれらのlossを線形に足したものです．なお，N_posは物体検出されたアンカーの数です．βは各lossを混ぜる際のパラメータで人間が設定します．

 

#### 評価手法
画像の物体検出と同様に，Average Precision(AP)を用います．mAPはクラスで平均した値です．なお，IoUは0.5に設定されています．

#### Slimmer Design.について
Point Pillarでは層設計を軽量化することで高速化を図りました．
先行研究のVoxelnetではPointNetを二回適用しているため処理速度が遅いという問題がありました．Point Pillarでは，PointNetの適用を一回にすることで，Voxelnetより2.5 ms高速になりました．
また，PointNet内の次元数を削減し4.5 ms高速化し，Detectionを担う層の次元数を下げて3.9 ms高速化しました．

#### 実装上の高速化
Pytorchで実装したが，NVIDIA TensorRT(GPU推論に最適化されたライブラリ) を使用することで，より最適化され，ピュアなPytorchよりも45.5%高速化できた．
JETSONを使用したエッジ側での推論が可能になる可能性がある．


物体検出を行うデータは、数１０ＧＢあるので、この演習では扱いません、以下に学習済みモデルを呼び出すサンプルコードを置きますので、参考にしてください。

` /path/to/your/dataset`をダウンロードした場所に指定すると、動きます。


In [ ]:
import os
import open3d.ml as _ml3d
import open3d.ml.tf  as ml3d

cfg_file = "ml3d/configs/pointpillars_kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model)
cfg.dataset['dataset_path'] = "/path/to/your/dataset"
dataset = ml3d.datasets.KITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="gpu", **cfg.pipeline)

# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"


In [ ]:
%cd "/content/Open3D-ML/logs"
!wget "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth"
!ls 



In [ ]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()